In [42]:
import numpy as np
import scipy.io
import matlab.engine
import matplotlib.pyplot as plt

In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from pathlib import Path

#### Load matlab data

In [33]:
feats = scipy.io.loadmat('features_out.mat')
labels = scipy.io.loadmat('labels_out.mat')

In [34]:
feats = feats['features']
labels = labels['position']

In [35]:
feats.shape, labels.shape

((48, 1, 1, 3), (3, 3))

#### Explanation of shape:

This example uses the magnitude of each multipath component in the CIR as training data. Therefore, the generated CIRs are real-valued. The example stores the CIRs in a four-dimensional array of size N
s
-by-N
tx-rx
-by-N
AP
-by-N
r
.

N
s
 is the number of time-domain samples in the CIR.

N
tx-rx
 is the total number of transmit-receive antenna pairs.

N
AP
 is the number of APs.

N
r
 is the number of channel realizations for all SNR points.

https://www.mathworks.com/help/wlan/ug/three-dimensional-indoor-positioning-with-802-11az-fingerprinting-and-deep-learning.html

#### Problem: Labels shape should have at least one dimension that matches the number of time samples

Going on the assumption that labels shape will be (time samples, num aps, 3). 3 for 3 dimensions for positioning.

In [36]:
# TEMPORARY UNTIL WE FIX THE LABELS
labels = np.random.random((feats.shape[0], 1))
# might need to scale up the sample in case there aren't enough features

In [62]:
X = torch.Tensor(feats)
Y = torch.Tensor(labels)

dataset = TensorDataset(X, Y)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

#### Define model

In [63]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.linear = nn.Linear(50, 1) # not 50, need to modify this
    
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.pool1(out)
        out = F.relu(self.conv2(out))
        out = self.pool2(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
        

In [64]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyCNN().to(device)

In [65]:
crit = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#### Training/Evaluating NN

In [ ]:
for epoch in range(10):
    train_loss = 0
    test_loss = 0

    
    model = model.train()
    
    for batch_idx, (ft, lbl) in enumerate(train_loader):
        ft, lbl = ft.to(device), lbl.to(device)
        optimizer.zero_grad()
        output = model(ft)
        loss = crit(output, lbl)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
        
    model = model.eval()
    
    with torch.no_grad():
        for batch_idx, (ft, lbl) in enumerate(test_loader):
            ft, lbl = ft.to(device), lbl.to(device)
            output = model(ft)
            loss = crit(output, lbl)
            test_loss += loss
            
    print('Epoch {} | Training loss = {} | Test loss = {}'.format(epoch, train_loss, test_loss))

In [68]:
# save model
model_dir = Path('./models')
model_dir.mkdir(parents=True, exist_ok=True)
torch.save(model.state_dict(), model_dir / '6gcnn.pth')